In [1]:
from jupyter_set_up import init_django

init_django('pictionary')

In [2]:
import numpy as np
import pickle
from pictionary.models import DrawingTrain, DrawingTest
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from datetime import datetime

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)

        x = torch.randn(28,28).view(-1,1,28,28)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 50)

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = np.prod(x[0].shape)
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

In [6]:
class MyModel:
    @staticmethod
    def create_X_y(model):
        temp = model.objects.all().values_list('picture', flat=True).order_by('id')
        y = model.objects.all().values_list('category', flat=True).order_by('id')
        print("Queryset done\t{}".format(datetime.now()))
        
        X = []
        for img in temp:
            X.append(pickle.loads(img))

        print("y len: {}\t{}".format(len(y), datetime.now()))
        return torch.Tensor(np.array(X)), torch.Tensor(np.array(y))

    def create_train_test(self):
        train_X, train_y = self.create_X_y(DrawingTrain)
        test_X, test_y = self.create_X_y(DrawingTest)
        return train_X, train_y, test_X, test_y


In [5]:
start_time = datetime.now()
NN = MyModel()
train_X, train_y, test_X, test_y = NN.create_train_test()
print("Duration: {}".format(datetime.now() - start_time))

Dataset len: 2800488
Duration: 0:00:01.139992
Train len: 1866992
Test len: 933496
Duration: 0:02:44.167256
for Duration: 0:14:47.202240


TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(cnn, train_X, train_y):
    BATCH_SIZE = 100
    EPOCHS = 50
    optimizer = optim.Adam(cnn.parameters(), lr=0.01)
    loss_function = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            batch_X = train_X[i:i + BATCH_SIZE].view(-1, 1, 28, 28)
            batch_y = train_y[i:i + BATCH_SIZE]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            cnn.zero_grad()
            outputs = cnn(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backword()
            optimizer.step()
        print("Epoch: {} Loss: {}".format(epoch, loss))


def test(cnn, test_X, test_y):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = cnn(test_X[i].view(-1, 1, 28, 28).to(device))[0]
            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1
    print("Accuracy: {}".format(round(correct / total, 3)))

In [28]:
start_time = datetime.now()
cnn = CNN().to(device)
train(cnn, train_X, train_y)
test(cnn, test_X, test_y)
print("Duration: {}".format(datetime.now() - start_time))

  0%|                                                                                                                                                        | 0/18670 [00:00<?, ?it/s]


RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Float'